### Some things about bitwise operations.

x & y

Does a "bitwise and". Each bit of the output is 1 if the corresponding bit of x AND of y is 1, otherwise it's 0. 

example:

2&2 = 2
       
        2 ---> 0010
          and
        2 ---> 0010
              ------ 
               0010 = 2      
2&1 = 0
        
        2 ---> 0010
          and
        1 ---> 0001
              ------ 
               0000 = 1


x | y
    
Does a "bitwise or". Each bit of the output is 0 if the corresponding bit of x AND of y is 0, otherwise it's 1. 
    
2&1 = 3
        
        2 ---> 0010
           or
        1 ---> 0001
              ------ 
               0011 = 3


To have the octants (0-7) in binary numbers in mind: 

    0000 = 0        0001 = 1        0010 = 2         0011 = 3

    0100 = 4        0101 = 5        0110 = 6         0111 = 7


## Constructing the tree

Let's start to define the problem and our rules to build the tree. Consider there are $n=100$ particles randomly scatterd in the domain $x$, $y$, $z$ $\in$ $\left[ 0, 1 \right]$, each of them is a source and target. To contain all the particles, we define a **root** cubic cell centered at $(0.5,0.5, 0.5)$ with a side length of $1$. For a cubic cell, the radius is the half of side length, thus the root cell's radius $r_r$ is $0.5$. Then we choose $n_{crit}=10$ as the threshold to split a cell.

In [2]:
import numpy
from treecode_helper import Point, Particle

In [3]:
n = 100          # number of particles
particles = [ Particle(m=1.0/n) for i in range(n) ]

n_crit = 10      # max number of particles in a single cell

### Defining the class : Cell

Since each non-empty cell is an instance which has some same properties (eg. cell's center coordinates, its parent, its children, its multipole, particles inside if it's a leaf), we first need to define a class for a cell, and we call this class "Cell". Every cell is an instance of **Class** "**Cell**", and all the instances are stored in a **list** called "**cells**". Therefore, the root cell is "cells[0]". For those who have not been exposed to object-oriented programming in python, check [this](http://www.tutorialspoint.com/python/python_classes_objects.htm) out as a quick guide.

<img src="image/cell_class.png">

The figure above shows the "content" of a cell element:
* $x_c$, $y_c$, $z_c$, $r_c$: center coordinates and radius give the geometry of the cell
* a leaf of a cell is a particle stored in the cell, each leaf corresponds to a particle index from $0$ to $n-1$, and $n_{leaf}$ is number of leaves in the cell
* **parent** is the index (in the cells list) of the corresponding parent. 
* **child** is the array that contains the indices (in the cells list) of the corresponding children.
* **nchild** is an 8-bit binary number, each digit represents one of the eight child octants. $1$ denotes non-empty child cell, and $0$ denotes empty child cell in that octant. For example: nchild=00010100 means the current cell only has non-empty child in the fourth and sixth octant.
* **multipole**: array of $10$ multipole terms of the cell

In [4]:
class Cell():
    '''
    The class for a cell
    
    Arguments
    ----------
        n_crit: maximum number of particles in a leaf cell.
        
    Attributes
    ----------
        nleaf (int): number of leaves in the cell.
        leaf (array of int): array of leaves indices.
        nchild (int): 8-bit binary number, used to keep
                      track of the empty child cells
        child (array of int): array of children indices.
        parent (int): index of parent cell. 
        x, y, z (float): coordinates of the cell's center.
        r (float): radius of the cell (half of the length for cubic cell)
        multipole (array of float): array of multipoles' cell.
    '''
    
    def __init__(self, n_crit):
        
        self.nleaf = 0
        self.leaf = numpy.zeros(n_crit, dtype=numpy.int)
        self.nchild = 0
        self.child = numpy.zeros(8, dtype=numpy.int)
        self.parent = 0
        self.x = self.y = self.z = 0.
        self.r = 0. 
        self.multipole= numpy.zeros(10, dtype=numpy.float)

In [5]:
#Let's generate the root cell

root = Cell(n_crit)
root.x, root.y, root.z = 0.5, 0.5, 0.5
root.r = 0.5

If we want to add more than 10 particles then we need to create a child by splitting the root cell. "Adding a cell" is going to be a dependency of "splitting a cell".

The new child will be append at the end of the cells list.

The relationships between parent and children are:
    
* $r_{child}$ = $\frac{1}{2}r_{parent}$ for a cubic cell.
* $x_{c_{c}}$, $y_{c_{c}}$, $z_{c_{c}}$ can be determined by its $octant$ and its parent's coordinates $x_{c_{p}}$, $y_{c_{p}}$, $z_{c_{p}}$.

We need to establish a mutual reference between the parent and the child in the cells list. Consider a new child is created in the parent's $5th$ octant. We assign the new child's index to the parent by "parent.child[$4$]=index_child", and assign the parent's index to the new child by "child.parent=index_parent". Don't forget the 8-bit binary marker "nchild". Since the new child is at $5th$ octant, the fifth digit from the right should be changed from "0" to "1". Recall that we always manipulate the binary number with bit shift.
